In [71]:
import pandas as pd
import numpy as np

## Task 2

In [72]:
result = pd.read_csv('assignments_from_pool_29960711__06-12-2021.tsv', sep='\t')

In [73]:
def link2id(x):
    return x.split('/')[-1].split('"')[0]

result['INPUT:image'] = result['INPUT:image'].apply(link2id)

In [74]:
result['GOLDEN:result'] = result['GOLDEN:result'].fillna(1.)

In [75]:
generated_img_result = result[result['GOLDEN:result'] == 1.0]

In [76]:
aggregated_result = generated_img_result.groupby('INPUT:image').sum()

In [77]:
final_result = aggregated_result[['OUTPUT:gender', 'OUTPUT:result', 'OUTPUT:confidence', 'GOLDEN:result']]

In [78]:
final_result = final_result.rename({
    'OUTPUT:gender': 'gender', 
    'OUTPUT:result': 'result',
    'OUTPUT:confidence': 'confidence', 
    'GOLDEN:result': 'n_workers'
}, axis=1)

In [79]:
final_result['n_workers'] = final_result['n_workers'].astype('int')

workers do not always agree on genders of the image: indicating random selections

In [80]:
def control_consensus(df):
    # male or female
    return sum((df['gender'] == df['n_workers']) | (df['gender'] == 0)) / len(df)

In [81]:
control_consensus(final_result)

0.912

images considered to be `real` by workers with more than 2 votings

In [82]:
# workers deem to be real
real_img = final_result[(final_result['n_workers'] - final_result['result'] >= 3)]

workers do not always agree with the gender

In [83]:
control_consensus(real_img)

0.9516129032258065

rate for successfully deceive more than 2 workers

In [84]:
n_real_imgs = len(real_img)
n_valid_imgs = len(final_result[final_result['n_workers'] >= 3])
f"decieved imgs: {n_real_imgs}; imgs with more than 3 workers: {n_valid_imgs} rate: {n_real_imgs / n_valid_imgs}"

'decieved imgs: 62; imgs with more than 3 workers: 424 rate: 0.14622641509433962'

In [36]:
f"concensus on gender: {control_consensus(real_img)}"

'concensus on gender: 0.9516129032258065'

### Create Task 3

In [47]:
real_img_result = result[result['GOLDEN:result'] == 0.]
real_img_result['n_workers'] = np.ones(len(real_img_result))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
aggregated_real_img_result = real_img_result.groupby('INPUT:image').sum()\
    [['OUTPUT:gender', 'OUTPUT:result', 'OUTPUT:confidence', 'n_workers']]

In [55]:
# workers deemed as fake
fake_img = aggregated_real_img_result[aggregated_real_img_result['OUTPUT:result'] >= 2]

In [56]:
task = pd.read_csv('tasks/tasks_for_pool_29747529__23-11-2021_training.tsv', sep='\t')

In [69]:
from random import random

new_data = task.iloc[0:0]
real_img_list = real_img.index.to_list()
fake_img_list = fake_img.index.to_list()

for i in range(len(real_img_list)):
    image_1 = "https://raw.githubusercontent.com/BirkhoffG/crowd_project/main/img/" + real_img_list[i]
    image_2 = "https://raw.githubusercontent.com/BirkhoffG/crowd_project/main/original/" + fake_img_list[i]
    if random() < 0.5:
        new_data = new_data.append({
            'INPUT:image_left': image_1, 'INPUT:image_right': image_2, 'GOLDEN:result': 'Right', 
            'GOLDEN:textfield': '', 'HINT:text': 'The image on the Right is REAL.'
        }, ignore_index=True)
    else:
        new_data = new_data.append({
            'INPUT:image_left': image_2, 'INPUT:image_right': image_1, 'GOLDEN:result': 'Left', 
            'GOLDEN:textfield': '', 'HINT:text': 'The image on the Left is REAL.'
        }, ignore_index=True)

In [70]:
new_data.to_csv('task3.tsv', sep='\t', index=None)